In [6]:
%reload_ext autoreload
%autoreload 2

In [7]:
#|export
DATASET_PATH = "/home/tiurin/projects/ExampleProject/Temp"

In [8]:
#|export
import cv2 as cv

In [9]:
#|export
def color_feature(image):
    return np.mean(image[image>0])

In [10]:
#|export
def moving_average_filter(img, size):
    kernel = np.ones((size, size), np.float32) / (size*size)
    filtered = cv.filter2D(img, -1, kernel)
    return filtered

In [11]:
#|export

def sobel(image):
    
    ddepth = cv.CV_16S
    scale = 1
    delta = 0
    
    grad_x = cv.Sobel(image, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    grad_y = cv.Sobel(image, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv.BORDER_DEFAULT)
    
    abs_grad_x = cv.convertScaleAbs(grad_x)
    abs_grad_y = cv.convertScaleAbs(grad_y)
    
    grad = cv.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)
    
    return grad

In [12]:
#|export
def focus_features(image):
    FM1 = np.mean(sobel(image))
    
    fm2 = np.mean(sobel(moving_average_filter(image, 3)))
    FM2 = FM1 - fm2
    
    fm3 = np.mean(sobel(moving_average_filter(image, 5)))
    FM3 = fm2 - fm3
    
    return FM1, FM2, FM3

In [13]:
#|export
def illumination_features(image):
    non_zero_vals = image[image>0]
    IM1 = np.mean(non_zero_vals)
    IM2 = np.var(non_zero_vals[non_zero_vals<=IM1])
    IM3 = np.var(non_zero_vals[non_zero_vals>IM1])    
    IM4 = np.var(non_zero_vals)
    return IM1, IM2, IM3, IM4

In [14]:
#|export
def contrast_features(image):
    hist = cv.calcHist([image.flatten()], [0], None, [16], [1, 256])
    ct1 = np.abs(hist/np.sum(hist)-0.0625).sum()
    ct2 = (hist==0).sum()
    
    avgd = moving_average_filter(image, 3)
    hist = cv.calcHist([avgd.flatten()], [0], None, [16], [1, 256])
    ct3 = np.abs(hist/np.sum(hist)-0.0625).sum()
    ct4 = (hist==0).sum()
    return ct1, ct2, ct3, ct4

In [15]:
#|export
def extract_sift_features(patches):
    sift_extractor = cv.SIFT_create()
    features = []
    for patch in patches:
        if patch is None:
            return None
        kp, des = sift_extractor.detectAndCompute(patch, None)
        if des is None: continue
        features.append(des)
    if len(features) == 0: return None
    features = np.concatenate(features, axis=0)
    return features

In [16]:
#|export
def downscale_image(image, max_dim):
    height, width = image.shape[:2]
    
    if height <= max_dim and width <= max_dim:
        return image
    
    aspect_ratio = float(height) / float(width)

    if height > width:
        new_height = max_dim
        new_width = int(new_height / aspect_ratio)
    else:
        new_width = max_dim
        new_height = int(new_width * aspect_ratio)

    resized_image = cv.resize(image, (new_width, new_height))
    return resized_image

In [17]:
#|export
def load_image(img, max_size=400):
    if type(img) is str:
        img = cv.imread(img, 0)
        
    if img is None: 
        return None
    
    img = downscale_image(img, max_size)
    return img

In [18]:
#|export
def get_image_features(img):
    img = load_image(img)
    features_ptchs = extract_sift_features([img])
    return features_ptchs

In [19]:
#|export
from joblib import Parallel, delayed

In [20]:
#|export
NUM_CORES = 29

In [21]:
#|export
import numpy as np
from sklearn.cluster import MiniBatchKMeans as KMeans
import glob
import os
import cv2 as cv
import tqdm
import pickle
import random

class FeatureGenerator():
    
    def __init__(self, dataset_path_bow=None, model_name_kmean=None, feature_size=100):
        
        self.feature_size = feature_size
        
        if os.path.isfile(model_name_kmean+"_kmeans.pkl"):
            with open(model_name_kmean+"_kmeans.pkl", "rb") as f:
                self.kmeans = pickle.load(f)
                print('Loaded kmeans model')
        else:
            print('Train kmeans model')
            
            data = []
            if not isinstance(dataset_path_bow, list):
                dataset_path_bow = [dataset_path_bow]

            for p in dataset_path_bow:
                imgs = [im for im in glob.glob(os.path.join(p, '*.png'))]
                data.extend(imgs)
                imgs = [im for im in glob.glob(os.path.join(p, '*.jpg'))]
                data.extend(imgs)
                imgs = [im for im in glob.glob(os.path.join(p, '*.jpeg'))]
                data.extend(imgs)
            
            descriptors = []
            descriptors = Parallel(n_jobs=NUM_CORES)(delayed(get_image_features)(img) for img in tqdm.tqdm(data))
            descriptors = [item for item in descriptors if item is not None]

            training_descriptors = np.concatenate(descriptors, axis=0)

            self.kmeans = KMeans(n_clusters=self.feature_size, 
                                 random_state=0, 
                                 batch_size=1024*NUM_CORES).fit(training_descriptors.astype('float'))
            
            with open(model_name_kmean+'_kmeans.pkl', "wb") as f:
                pickle.dump(self.kmeans, f)
        
    def generate_features(self, img, include_generic=True):
        
        features_ptchs = get_image_features(img)
        
        histogram = np.zeros(self.feature_size)
        if features_ptchs is not None:
            for i in range(features_ptchs.shape[0]):
                index = self.kmeans.predict([features_ptchs[i].astype('float')])[0]
                histogram[index] += 1
        
        if include_generic:
            histogram = np.append(histogram, color_feature(img))
            histogram = np.append(histogram, focus_features(img))
            histogram = np.append(histogram, contrast_features(img))
            histogram = np.append(histogram, illumination_features(img))
        
        return histogram
    

In [22]:
dataset_path_bow = [os.path.join(DATASET_PATH, 'processed', '0'), 
                    os.path.join(DATASET_PATH, 'processed_test', '0'), 
                    os.path.join(DATASET_PATH, 'processed_test', '1'), 
                    os.path.join(DATASET_PATH, 'processed_test', '1'), 
                    os.path.join(DATASET_PATH, 'processed_test', '2'), 
                    os.path.join(DATASET_PATH, 'processed_test', '2')]
dataset_path_bow.extend([im for im in glob.glob(os.path.join(DATASET_PATH, 'trash_images',  '**/*'), recursive=True) if '.' not in im])

In [23]:
%%time
feature_gen = FeatureGenerator(dataset_path_bow, 'all_data')

Loaded kmeans model
CPU times: user 2.43 ms, sys: 31.8 ms, total: 34.3 ms
Wall time: 33.2 ms


In [24]:
#|export
def zoom_augmentation(image, zoom_factor):
    height, width = image.shape[:2]
    new_height, new_width = int(height * zoom_factor), int(width * zoom_factor)
    resized_image = cv.resize(image, (new_width, new_height))
    top_pad = int((height - new_height) / 2)
    bottom_pad = height - new_height - top_pad
    left_pad = int((width - new_width) / 2)
    right_pad = width - new_width - left_pad
    padded_image = cv.copyMakeBorder(resized_image, top_pad, bottom_pad, 
                                     left_pad, right_pad, cv.BORDER_CONSTANT, value=0)
    return padded_image

In [25]:
#|export
def random_rotation_augmentation(image, angle_range=(-180, 180)):
    # Randomly choose rotation angle
    angle = random.uniform(*angle_range)
    
    # Get image size
    height, width = image.shape[:2]
    
    # Compute rotation matrix
    rotation_matrix = cv.getRotationMatrix2D((width / 2, height / 2), angle, 1.0)
    
    # Perform rotation
    rotated_image = cv.warpAffine(image, rotation_matrix, (width, height))
    
    return rotated_image

In [26]:
#|export
def augument_image(img):
    zoom = random.uniform(0.8, 1)
    img = zoom_augmentation(img, zoom)
    img = random_rotation_augmentation(img)
    return img

In [27]:
#|export
import hashlib

class Dataset():
    def __init__(self, feature_generator):
        self.feature_gen = feature_generator
        self.data = []
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
    def load_data(self, data_paths):
        random.seed(42)
        random.shuffle(data_paths)
        sample = data_paths[0]
        x = self.prepare_sample(sample[1], sample[0], False)
        hash_str = str(x[0])+str(x[1])+str(x[2])
        hash_object = hashlib.md5(hash_str.encode())
        model_name = str(hash_object.hexdigest())
        
        if os.path.isfile(model_name+"_data.pkl"):
            with open(model_name+"_data.pkl", "rb") as f:
                self.data = pickle.load(f)
                print('Loaded data')
        else:
            print('Generating data')
            
            result_list = Parallel(n_jobs=NUM_CORES)(delayed(self.prepare_sample)(img_pth, class_id, False) for class_id, img_pth in tqdm.tqdm(data_paths))
            result_list = [r for r in result_list if r[0] is not None and r[1] is not None]
            self.data.extend(result_list)
            
            with open(model_name+'_data.pkl', "wb") as f:
                pickle.dump(self.data, f)
    
    def prepare_sample(self, img_pth, class_id, augument=True):
        img = load_image(img_pth)
        if img is None:
            return (None, None, None)
        if augument:
            img = augument_image(img)
        features = self.feature_gen.generate_features(img)
        return (class_id, features, img_pth)

In [28]:
#|export
import os, glob
import random

class EyeQDataset(Dataset):

    def __init__(self, dataset_path, feature_gen):
        super().__init__(feature_gen)
        data_paths = []
        
        if not isinstance(dataset_path, list):
            dataset_path = [dataset_path]
        
        for p in dataset_path:
            for i in range(3):
                class_i = [(2-i, im) for im in glob.glob(os.path.join(p, str(i), '*.png'))]
                data_paths.extend(class_i)
        
        self.load_data(data_paths)
    

In [29]:
#|export
import os, glob
import random

class RandomImages(Dataset):

    def __init__(self, dataset_path, feature_gen):
        super().__init__(feature_gen)
        data_paths = []
        
        if not isinstance(dataset_path, list):
            dataset_path = [dataset_path]
            
        for p in dataset_path:
            class_i = [(0,im) for im in glob.glob(os.path.join(p,  '**/*.jpg'), recursive=True)]
            data_paths.extend(class_i)
            class_i = [(0,im) for im in glob.glob(os.path.join(p,  '**/*.jpeg'), recursive=True)]
            data_paths.extend(class_i)
        
        self.load_data(data_paths)
        

In [30]:
#|export
class DRIMDBDataset(Dataset):

    def __init__(self, dataset_path, feature_gen):
        super().__init__(feature_gen)
        data_paths = []
        
        bad_samples = [(0,im) for im in glob.glob(os.path.join(dataset_path, 'Bad', '*.jpg'))]
        data_paths.extend(bad_samples)
        
        good_samples = [(2,im) for im in glob.glob(os.path.join(dataset_path, 'Good', '*.jpg'))]
        data_paths.extend(good_samples)
        
        outlier_samples = [(0, im) for im in glob.glob(os.path.join(dataset_path, 'Outlier', '*.jpg'))]
        data_paths.extend(outlier_samples)
        
        self.load_data(data_paths)
        

In [31]:
dataset = EyeQDataset([os.path.join(DATASET_PATH, 'processed'), os.path.join(DATASET_PATH, 'processed_test')], feature_gen)

Loaded data


In [32]:
dataset_random = RandomImages(os.path.join(DATASET_PATH, 'trash_images'), feature_gen)

Loaded data


In [33]:
dataset_drimdb = DRIMDBDataset('/home/tiurin/projects/ExampleProject/Temp/DRIMDB', feature_gen)

Loaded data


In [36]:
dataset_drimdb[10][-1]

'/home/tiurin/projects/ExampleProject/Temp/DRIMDB/Good/drimdb_good (65).jpg'

In [1]:
#|export
from sklearn.model_selection import train_test_split

def get_dataset(eye=True, random=True, test=False, test_size=0.0, features_range=(0, 112)):
    X = []
    y = []
    
    feature_gen = FeatureGenerator([], 'all_data')
    
    if eye:
        dataset = EyeQDataset([os.path.join(DATASET_PATH, 'processed'), os.path.join(DATASET_PATH, 'processed_test')], feature_gen)
        X.extend([d[1][features_range[0]:features_range[1]] for d in dataset.data])
        y.extend([d[0] for d in dataset.data])
    if random:
        dataset_random = RandomImages(os.path.join(DATASET_PATH, 'trash_images'), feature_gen)
        X.extend([d[1][features_range[0]:features_range[1]] for d in dataset_random.data])
        y.extend([d[0] for d in dataset_random.data])
    if test:
        dataset_drimdb = DRIMDBDataset('/home/tiurin/projects/ExampleProject/Temp/DRIMDB', feature_gen)
        X.extend([d[1][features_range[0]:features_range[1]] for d in dataset_drimdb.data])
        y.extend([d[0] for d in dataset_drimdb.data])
    
    if test_size > 0:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
        return X_train, X_test, y_train, y_test
    else:
        return X, y

In [37]:
import os
ipynb_path = os.path.join(os.getcwd(), 'dataset.ipynb')
ipynb_path

'/share/data_rechenknecht03_2/students/tiurin/projects/ExampleProject/Temp/code/nbs/dataset.ipynb'

In [39]:
#|default_exp dataset
#|hide
import nbdev; nbdev.nbdev_export(ipynb_path)